<h3>Antes de rodar esse código, certifique que você tenha o Pycharm, SAP e autorização para instalação e acesso via script no SAP.</h3>

In [ ]:
# List all non-standard packages to be imported by your
# script here (only missing packages will be installed)
from ayx import Package
#Package.installPackages(['pandas','numpy'])

In [ ]:
from ayx import Alteryx

In [ ]:
#Input com os dados do Alteryx para o Panda

entrada1 = Alteryx.read("#1") #Dados de Itens de Oportunidades
entrada2 = Alteryx.read("#2") #Diretório raiz do arquivo

In [ ]:
#Salva os dados do input no clipboard para execução no SAP GUI

import pandas

pandas.DataFrame(entrada1).to_clipboard(header=False, index=False)

In [ ]:
#Tratamento de dados do input com o diretório raiz onde o arquivo de execução do Alteryx se encontra

entradiretorio = pandas.DataFrame(entrada2).to_string(header=False, index=False)
diretorio = "C:\\Users\\kaique.souza\\Desktop\\Volumetria\\"

In [ ]:
import win32com.client, sys, subprocess, time


def saplogin():

    try:

        #Busca o aplicativo do SAP GUI para execução (Desativado)
        #path = r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe"
        #subprocess.Popen(path)
        #time.sleep(10)

        SapGuiAuto = win32com.client.GetObject('SAPGUI')
        if not type(SapGuiAuto) == win32com.client.CDispatch:
            return

        application = SapGuiAuto.GetScriptingEngine
        if not type(application) == win32com.client.CDispatch:
            SapGuiAuto = None
            return

        connection = application.Children(0) #OpenConnection("JNP - SRM de Produção V04", True)
        if not type(connection) == win32com.client.CDispatch:
            application = None
            SapGuiAuto = None
            return

        session = connection.Children(0)
        if not type(session) == win32com.client.CDispatch:
            connection = None
            application = None
            SapGuiAuto = None
            return

        #Faz logon no SAP (Desativado)
        #session.findById("wnd[0]/usr/txtRSYST-BNAME").text = "USUARIO"
        #session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = "SENHA"
        #session.findById("wnd[0]").sendVKey(0)

        #Script do SAP GUI
        session.findById("wnd[0]").maximize
        session.findById("wnd[0]/tbar[0]/okcd").text = "/NSE16"
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]/usr/ctxtDATABROWSE-TABLENAME").text = "YP_V_PDIGP"
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]/mbar/menu[3]/menu[0]/menu[1]").select()
        session.findById("wnd[1]").sendVKey(14)
        session.findById("wnd[1]/usr/chk[1,4]").selected = True
        session.findById("wnd[1]/usr/chk[1,16]").selected = True
        session.findById("wnd[1]/usr/chk[1,16]").setFocus
        session.findById("wnd[1]").sendVKey(6)
        session.findById("wnd[0]/usr/btn%_I1_%_APP_%-VALU_PUSH").press()
        session.findById("wnd[1]").sendVKey(24)
        session.findById("wnd[1]").sendVKey(8)
        session.findById("wnd[0]").sendVKey(8)
        session.findById("wnd[0]").sendVKey(43)
        session.findById("wnd[1]").sendVKey(0)
        session.findById("wnd[1]/usr/ctxtDY_PATH").text = diretorio
        session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "YP_V_PDIGP.XLSX"
        session.findById("wnd[1]").sendVKey(11)
        session.findById("wnd[0]").sendVKey(3)
        session.findById("wnd[0]").sendVKey(3)
        session.findById("wnd[0]").sendVKey(3)

    except Exception as e:
        print(f"Erro: {e}")


    finally:
        session = None
        connection = None
        application = None
        SapGuiAuto = None

saplogin()


In [ ]:
#Realiza a leitura dos arquivos em Excel que o SAP extraiu

arquivo1 = pandas.read_excel(diretorio + "YP_V_PDIGP.xlsx", sheet_name='Sheet1', engine='openpyxl')

In [ ]:
#Exporta dos dados para os outputs

Alteryx.write(arquivo1,1)